### Transformer Decoder

<div align="center">
  <img src="https://res.cloudinary.com/edlitera/image/upload/c_fill,f_auto/v1680629118/blog/gz5ccspg3yvq4eo6xhrr" alt="Transformer Decoder" width="300">
</div>

In [2]:
import torch
import torch.nn as nn

In [ ]:
# Importing classes from the respective notebooks
%run 4_Multihead_Attention.ipynb
%run 5_FeedForward.ipynb

In [3]:
class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim, dropout=0.1):
        """
        Single transformer decoder layer with masked self-attention, 
        cross-attention, and feed-forward network.
        
        Args:
            embed_dim: Dimension of embeddings
            num_heads: Number of attention heads
            ff_hidden_dim: Hidden dimension of feed-forward network
            dropout: Dropout probability
        """
        super().__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        self.ff = FeedForward(embed_dim, ff_hidden_dim, dropout)
        self.norm3 = nn.LayerNorm(embed_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, self_attn_mask=None, cross_attn_mask=None):
        """
        Forward pass of decoder layer.
        
        Args:
            x: Input tensor [batch_size, seq_len, embed_dim]
            enc_output: Encoder output [batch_size, enc_seq_len, embed_dim]
            self_attn_mask: Mask for self-attention (usually causal mask)
            cross_attn_mask: Mask for cross-attention
            
        Returns:
            x: Output tensor [batch_size, seq_len, embed_dim]
        """
        # Self-attention block with residual connection and layer normalization
        residual = x
        x = self.norm1(x)
        x = residual + self.dropout(self.self_attn(x, x, x, self_attn_mask))
        
        # Cross-attention block with residual connection and layer normalization
        residual = x
        x = self.norm2(x)
        x = residual + self.dropout(self.cross_attn(x, enc_output, enc_output, cross_attn_mask))
        
        # Feed-forward block with residual connection and layer normalization
        residual = x
        x = self.norm3(x)
        x = residual + self.dropout(self.ff(x))
        
        return x